# Word2Vec Generator

## Import Library

In [11]:
import tensorflow as tf
import pandas as pd
import numpy as np
import joblib

## Import Model Embedding

In [3]:
embedding = tf.keras.models.load_model("../../model/nnlm-id-dim50/")

2023-06-24 11:47:26.505654: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-06-24 11:47:26.505889: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-06-24 11:47:26.506020: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (SAT-PC): /proc/driver/nvidia/version does not exist
2023-06-24 11:47:26.506876: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
def embed_text(model : object, 
               text : str) -> list :
    """Function to create embedding 
        using model object

    Args:
        model (object): Embedding model
        text (str): text input

    Returns:
        list: embedding of the text input
    """
    try :
        embedding_list = model([text]).numpy().squeeze().tolist()
        return embedding_list
    except : 
        print(f"The error text is : {text}")
        return None
    

## Vetorize Data

Data Source : 
- https://github.com/kirralabs/indonesian-NLP-resources
- https://github.com/sastrawi/sastrawi/blob/master/data/kata-dasar.original.txt
- https://github.com/edwardsamuel/Wilayah-Administratif-Indonesia/tree/master/csv

In [9]:
# import data
district_data = pd.read_csv("../../data/raw/districts.csv", header=None)
province_data = pd.read_csv("../../data/raw/provinces.csv", header=None)
regencies_data = pd.read_csv("../../data/raw/regencies.csv", header=None)
villages_data = pd.read_csv("../../data/raw/villages.csv", header=None)
kata_data = pd.read_table("../../data/raw/kata-dasar.original.txt", header=None)

# format data
district_data = district_data[2].apply(lambda x: x.lower())
province_data = province_data[1].apply(lambda x: x.lower())
regencies_data = regencies_data[2].apply(lambda x: x.lower())
villages_data = villages_data[2].apply(lambda x: x.lower())

# concat data
dataset = pd.concat([district_data, province_data, regencies_data, villages_data, kata_data], axis=0)
dataset.columns = ["words"]
dataset.drop_duplicates(inplace=True)
dataset.dropna(inplace=True)
dataset.reset_index(drop=True, inplace=True)
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93566 entries, 0 to 93565
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   words   93566 non-null  object
dtypes: object(1)
memory usage: 731.1+ KB


In [10]:
# create embedding 
dataset['embedding'] = dataset['words'].apply(lambda x : embed_text(model=embedding, text=x))

In [14]:
joblib.dump(dataset,"../../data/processed/word_embedding.pkl")

['../../data/processed/word_embedding.pkl']